In [ ]:
import boto3
import numpy as np
from astropy.io import fits
import gzip
import re
from io import BytesIO
import io
import pandas as pd 
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv("/home/bekah/glcat/flats/lds749b.csv")
df

In [ ]:
s3 = boto3.client('s3')

bucket_name = 'backplanetest'

def download_fits(bucket_name, key):
    try:
            obj = s3.get_object(Bucket=bucket_name, Key=key)
            df = pd.read_csv(obj['Body'])
            
            # target is lds 749b 
            target_ra = 323.06763626692
            target_dec = 0.25399979666

            df['RA_distance'] = abs(df['ra'] - target_ra)
            df['DEC_distance'] = abs(df['dec'] - target_dec)
        
            closest_row = df.loc[(df['RA_distance'] + df['DEC_distance']).idxmin()]
            
            return closest_row     
        
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
        
def get_exptime(bucket_name, key): 
    try:
            obj = s3.get_object(Bucket=bucket_name, Key=key)
            fits_data = obj['Body'].read()
            fits_file = fits.open(io.BytesIO(fits_data))
            header = fits_file[1].header
            print(header)
            exptime = header['EXPT_0']
            start = header['EXPSTART']
            fits_file.close()
            return start, exptime
        
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None, None
        else:
            print("something else went wrong")
            print(str(e))
            return None, None
        
        
def calculate_magnitude(expt, flux, b):
    if b == 'n': 
        scale = 20.08 
    elif b == 'f':
        scale = 18.82
    else: 
        print("b is not a valid band")
        return None 
    ab_mag = -2.5 * np.log10(flux / expt) + scale

    return ab_mag



In [ ]:
# iterate through white dwarf list, downloading data for the star in each leg of each eclipse 
# add eclipse data & star data together 
lds749b_list = []
b = 'f'

for i in range(len(df)): 
    e = df.iloc[i]['eclipse'] 
    legs = df.iloc[i]['legs'] 
    eclipse = str(e).zfill(5)
    
    for leg in range(legs):
        data = download_fits(bucket_name, f"e{eclipse}-{b}d-lds749b/e{eclipse}-{b}d-full-{leg}-photom-12_8.csv")
        start, exptime = get_exptime(bucket_name,  f"e{eclipse}-{b}d-lds749b/e{eclipse}-{b}d-full-{leg}-rice.fits")
        if data is not None: 
            data['eclipse'] = eclipse
            data['leg'] = leg
            data['exptime'] = exptime
            data['start_time'] = start
            abmag = calculate_magnitude(data['exptime'], data['flux'], b)
            data['ab_mag'] = abmag
            print(f"mag is {abmag} for leg {leg} in eclipse {eclipse}, this is run {i}")
            lds749b_list.append(data)

lds749b = pd.DataFrame(lds749b_list, index=None)

In [ ]:
exptime

In [ ]:
eclipse

In [ ]:
lds749b.to_csv("lds749b_fuv_2.csv")

In [ ]:
df.iloc[0]['legs']

In [ ]:
plt.scatter(lds749b['xcentroid'],lds749b['ycentroid'], c=lds749b['ecl'],s=.3)
plt.xlabel("x centroid")
plt.ylabel("y centroid")
plt.title("CAI in image coordinates")
plt.colorbar()

In [ ]:
split_cols = lds749b['eclipse'].str.split(r'\s+', expand=True)
lds749b['ecl'] = split_cols[0]
lds749b = lds749b.drop(columns=['eclipse'])
lds749b['ecl'] = pd.to_numeric(lds749b['ecl'])

In [ ]:
plt.scatter(lds749b['eclipse'],lds749b['ab_mag'],s=.5)

In [ ]:
plt.scatter(lds749b['ecl'],lds749b['peak'],s=.5)

In [ ]:
max(lds749b['ycentroid'])

In [ ]:
from scipy.interpolate import griddata
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
lds749b_full = pd.read_csv("lds749b_fuv_2.csv")

In [ ]:
lds749b_full.keys()

In [ ]:
lds749b = lds749b_full[lds749b_full['ecl']==17497]

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b[['xcentroid', 'ycentroid']].values
values = lds749b['ab_mag'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower')
plt.scatter(lds749b['xcentroid'], lds749b['ycentroid'], c=lds749b['ab_mag'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated magnitude')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points')
plt.show()

In [ ]:
group_sizes = lds749b_full.groupby('ecl').size()

sorted_group_sizes = group_sizes.sort_values(ascending=False)
full12 = sorted_group_sizes[sorted_group_sizes>=]

In [ ]:
import pyarrow.parquet as parquet

aspect = parquet.read_table("/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet").to_pandas()

In [ ]:


def rotate_point_around_center(x, y, center_x, center_y, eclipse):
    
    roll = aspect[aspect['eclipse']==eclipse].iloc[0]['roll']
    
    angle_radians = np.radians(roll)

    rotation_matrix = np.array([
    [np.cos(angle_radians), -np.sin(angle_radians)],
    [np.sin(angle_radians),  np.cos(angle_radians)]
])
    
    translated_x = x - center_x
    translated_y = y - center_y
    
    rotated_x, rotated_y = rotation_matrix.dot([translated_x, translated_y])
    
    final_x = rotated_x + center_x
    final_y = rotated_y + center_y
    
    return final_x, final_y

lds749b_full[['xcentroid_rotated', 'ycentroid_rotated']] = lds749b_full.apply(
    lambda row: pd.Series(rotate_point_around_center(
        row['xcentroid'], row['ycentroid'], 1500, 1500, row['eclipse']
    )), axis=1
)



In [ ]:
lds749b_full = lds749b_full[(lds749b_full["ab_mag"]>15) & (lds749b_full["ab_mag"] <18)]

lds749b_full[["peak", "mag", "flux", "aperture_sum", "aperture_sum_edge"]].describe()


In [ ]:
lds749b_full.keys()

In [ ]:
 lds749b_full['real_ab_mag'] = -2.5 * np.log10( lds749b_full['aperture_sum'] / lds749b_full['exptime']) + 18.82

In [ ]:
lds749b_full['distance_from_center'] = np.sqrt((lds749b_full['xcentroid_rotated'] - 1500) ** 2 + (lds749b_full['ycentroid_rotated'] - 1500) ** 2)

In [ ]:
plt.scatter(lds749b_full['distance_from_center'],lds749b_full['real_ab_mag'],s=.5)
plt.xlabel('dist from center')
plt.ylabel('ab mag')
plt.title('all lds 749b for FUV')

In [ ]:
plt.scatter(lds749b_full['exptime'],lds749b_full['real_ab_mag'],s=.5)
plt.xlabel('expt')
plt.ylabel('mag')
plt.title('all lds 749b for FUV')

In [ ]:
plt.scatter(lds749b_full['start_time'],lds749b_full['exptime'],s=.5)

In [ ]:
plt.scatter(lds749b_full['real_ab_mag'],lds749b_full['aperture_sum'],s=1)
plt.xlabel('abmag')
plt.ylabel('ap sum')
plt.title('all lds 749b for FUV')

In [ ]:
plt.scatter(lds749b_full['real_ab_mag'],lds749b_full['peak'],s=1)

In [ ]:
#plt.scatter(lds749b_full['xcentroid_rotated'], lds749b_full['ycentroid_rotated'],c=lds749b_full['mag'])
plt.scatter(lds749b_full['xcentroid'], lds749b_full['ycentroid'],c=lds749b_full['ab_mag'], s=.75, cmap='grey')

In [ ]:
plt.scatter(lds749b_full['xcentroid_rotated'], lds749b_full['ycentroid_rotated'],c=lds749b_full['real_ab_mag'],cmap='grey', s=1)
plt.colorbar()

In [ ]:
plt.hist(lds749b_full['real_ab_mag'],bins=60)

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_full[['xcentroid', 'ycentroid']].values
values = lds749b_full['mag'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower', cmap='gray')
plt.scatter(lds749b_full['xcentroid'], lds749b_full['ycentroid'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated magnitude')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -7.5 mag, NUV')
plt.set_cmap('gray') 
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_full[['xcentroid_rotated', 'ycentroid_rotated']].values
values = lds749b_full['real_ab_mag'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower', cmap='gray')
plt.scatter(lds749b_full['xcentroid_rotated'], lds749b_full['ycentroid_rotated'], edgecolor='k', s=.1)
plt.colorbar(label='real_ab_mag')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -7.5 mag, NUV')
plt.set_cmap('gray') 
plt.show()

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(18, 18))
axs = axs.flatten() 


for i in range(20): 
    
    eclipse = full12.index[i]
    
    lds749b = lds749b_full[lds749b_full['ecl']==eclipse]
    
    grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

    points = lds749b[['xcentroid', 'ycentroid']].values
    values = lds749b['real_abmag'].values
    grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

    im = axs[i].imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower')
    axs[i].scatter(lds749b['xcentroid'], lds749b['ycentroid'], c=lds749b['real_abmag'], edgecolor='k', s=.1)
    axs[i].set_title(eclipse)

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(18, 18))
axs = axs.flatten() 


for i in range(20): 
    
    eclipse = full12.index[i]
    
    lds749b = lds749b_full[lds749b_full['ecl']==eclipse]
    
    grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

    
    lds749b[['xcentroid_rotated', 'ycentroid_rotated']] = lds749b.apply(
    lambda row: pd.Series(rotate_point_around_center(
        row['xcentroid'], row['ycentroid'], 1500, 1500, row['ecl']
    )), axis=1)
    
    points = lds749b[['xcentroid_rotated', 'ycentroid_rotated']].values
    values = lds749b['mag'].values
    grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

    im = axs[i].imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower')
    axs[i].scatter(lds749b['xcentroid_rotated'], lds749b['ycentroid_rotated'], 
                   c=lds749b['mag'], edgecolor='k', s=.1)
    axs[i].set_title(eclipse)

plt.tight_layout()
plt.show()



In [ ]:
full12

In [ ]:
## FUV STATISTICS 
lds749b_fuv = pd.read_csv("lds749b_fuv.csv")

In [ ]:

lds749b_fuv[["peak", "mag", "flux", "aperture_sum", "aperture_sum_edge"]].describe()


In [ ]:
lds749b_fuv = lds749b_fuv[lds749b_fuv["mag"]<-6.5]
lds749b_fuv[["peak", "mag", "flux", "aperture_sum", "aperture_sum_edge"]].describe()


In [ ]:
group_sizes = lds749b_fuv.groupby('ecl').size()
sorted_group_sizes = group_sizes.sort_values(ascending=False)
full12 = sorted_group_sizes[sorted_group_sizes>=12]

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(18, 18))
axs = axs.flatten() 


for i in range(20): 
    
    eclipse = full12.index[i]
    
    lds749b = lds749b_fuv[lds749b_fuv['ecl']==eclipse]
    
    grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

    points = lds749b[['xcentroid', 'ycentroid']].values
    values = lds749b['flux'].values
    grid_z = griddata(points, values, (grid_x, grid_y), method='linear')

    im = axs[i].imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower')
    axs[i].scatter(lds749b['xcentroid'], lds749b['ycentroid'], c=lds749b['flux'], edgecolor='k', s=.1)
    axs[i].set_title(eclipse)

plt.tight_layout()
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_fuv[['xcentroid', 'ycentroid']].values
values = lds749b_fuv['mag'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower')
plt.scatter(lds749b_fuv['xcentroid'], lds749b_fuv['ycentroid'], c=lds749b_fuv['mag'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated magnitude')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -6.5 mag, FUV')
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_fuv[['xcentroid', 'ycentroid']].values
values = lds749b_fuv['mag'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower', cmap='gray')
plt.scatter(lds749b_fuv['xcentroid'], lds749b_fuv['ycentroid'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated magnitude')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -6.5 mag, FUV')
plt.set_cmap('gray') 
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_fuv[['xcentroid', 'ycentroid']].values
values = lds749b_fuv['flux'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower', cmap='gray')
plt.scatter(lds749b_fuv['xcentroid'], lds749b_fuv['ycentroid'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated flux')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -6.5 mag, FUV')
plt.set_cmap('gray') 
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[0:3000:1000j, 0:3000:1000j]

points = lds749b_fuv[['xcentroid', 'ycentroid']].values
values = lds749b_fuv['peak'].values
grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

plt.imshow(grid_z.T, extent=(0, 3000, 0, 3000), origin='lower', cmap='gray')
plt.scatter(lds749b_fuv['xcentroid'], lds749b_fuv['ycentroid'], edgecolor='k', s=.1)
plt.colorbar(label='Interpolated peak')
plt.xlabel('X Centroid')
plt.ylabel('Y Centroid')
plt.title('Interpolated Field of Points without anything above -6.5 mag, FUV')
plt.set_cmap('gray') 
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
from scipy.stats import describe

points = lds749b_fuv[['xcentroid', 'ycentroid']].values

db = DBSCAN(eps=50, min_samples=3).fit(points) 
labels = db.labels_

lds749b_fuv['cluster'] = labels

cluster_stats_df = lds749b_fuv.groupby('cluster').describe()

In [ ]:
plt.scatter(cluster_stats_df['xcentroid']['50%'],cluster_stats_df['ycentroid']['50%'],c=cluster_stats_df['mag']['50%'],cmap='grey')
plt.colorbar()
plt.title("FUV clusters with median mag")

In [ ]:
plt.scatter(cluster_stats_df['xcentroid']['50%'],cluster_stats_df['ycentroid']['50%'],c=cluster_stats_df['peak']['50%'],cmap='grey')
plt.colorbar()
plt.title("FUV clusters with median peak")

In [ ]:
plt.scatter(cluster_stats_df['xcentroid']['50%'],cluster_stats_df['ycentroid']['50%'],c=cluster_stats_df['flux']['50%'],cmap='grey')
plt.colorbar()
plt.title("FUV clusters with median flux")

In [ ]:
plt.scatter(cluster_stats_df['xcentroid']['mean'],cluster_stats_df['ycentroid']['mean'],c=cluster_stats_df['flux']['std'],cmap='grey')
plt.colorbar()
plt.title("FUV clusters with std flux")

In [ ]:
plt.scatter(cluster_stats_df['xcentroid']['mean'],cluster_stats_df['ycentroid']['mean'],c=cluster_stats_df['mag']['std'],cmap='grey')
plt.colorbar()
plt.title("FUV clusters with std mag")

In [ ]:
plt.scatter(lds749b_fuv['xcentroid'],lds749b_fuv['ycentroid'],s=1)
plt.title("FUV CAI LDS 749B locations in image coords")

In [ ]:
plt.scatter(lds749b_full['xcentroid'],lds749b_full['ycentroid'],s=1)
plt.title("NUV CAI LDS 749B locations in image coords")

In [ ]:
plt.scatter(lds749b_full['ecl'],lds749b_full['mag'],s=1)
plt.xlabel('eclipse')
plt.ylabel('magnitude')

In [ ]:
from astropy.io import fits


In [ ]:
e6713 = fits.open("/home/bekah/Downloads/e06713-nd-b00-ffull-image-r.fits")

In [ ]:
plt.imshow(e6713[1].data)

In [ ]:
e6713[2].header

In [ ]:
data = e6713[2].data  
header = e6713[2].header  

hdu = fits.PrimaryHDU(data, header=header)
hdul_new = fits.HDUList([hdu])

hdul_new.writeto('e6713_flag_n.fits', overwrite=True)